In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive
import ipywidgets as widgets
import numpy as np
import re
from IPython.core.display import HTML, display


date = re.sub("-","",pd.to_datetime('today').strftime('%d-%m-%Y'))
txt_ini = "https://datosabiertos.bogota.gov.co/dataset/44eacdb7-a535-45ed-be03-16dbbea6f6da/resource/b64ba3c4-9e41-41b8-b3fd-2da21d627558/download/osb_enftransm-covid-"
txt_fini = "-.csv"
url = txt_ini + date + txt_fini
df0 = pd.read_csv(url,encoding='latin1', error_bad_lines=False, sep =";")
df = df0.drop([len(df0)-1,len(df0)-2,len(df0)-3,len(df0)-4])


'''Se excluyen los pacientes asintomaticos'''
df = df[df["Fecha de inicio de síntomas"] != "Asintomático"]


'''Se reasignan los indices'''
df.index = list(range(len(df)))


'''Se ajustan los registros de la variables Fecha de diagnostico para que el numero de dia tenga dos digitos'''
for i in range(len(df["Fecha de diagnóstico"])):
    if len(df["Fecha de diagnóstico"][i]) < 10:
        df["Fecha de diagnóstico"][i] = "0" + df["Fecha de diagnóstico"][i]
    else:
        df["Fecha de diagnóstico"][i] = df["Fecha de diagnóstico"][i]

        
'''Se eliminan los registros con valores NaN'''
df = df.dropna()


'''Se asignan valores a los indices'''
df.index = list(range(len(df)))


'''Se ajustan los registros de la variables Fecha de inicio de sintomas para que el numero de dia tenga dos digitos'''
for e in range(len(df["Fecha de inicio de síntomas"])):
    if len(df["Fecha de inicio de síntomas"][e]) < 10:
        df["Fecha de inicio de síntomas"][e] = "0" + df["Fecha de inicio de síntomas"][e]
    else:
        df["Fecha de inicio de síntomas"][e] = df["Fecha de inicio de síntomas"][e]


'''Se ajustan datos para que la fecha de la variable Fecha de diagnostico sea dd/mm/yyyy'''
from datetime import datetime
for a in range(len(df["Fecha de diagnóstico"])):
    df["Fecha de diagnóstico"][a] = datetime.strptime(df["Fecha de diagnóstico"][a], '%d/%m/%Y')


'''Se ajustan datos para que la fecha de la variable Fecha de inicio de sintomas sea dd/mm/yyyy'''
for o in range(len(df["Fecha de inicio de síntomas"])):
    df["Fecha de inicio de síntomas"][o] = datetime.strptime(df["Fecha de inicio de síntomas"][o], '%d/%m/%Y')


'''Se crea nueva variable y se pobla con la diferencia entre la fecha de diagnostico y la fecha de sintomas'''
df["Dias en Diagnosticar"] = [0] * df.shape[0]
for u in range(len(df["Dias en Diagnosticar"])):
    df["Dias en Diagnosticar"][u] = (df["Fecha de diagnóstico"][u] - df["Fecha de inicio de síntomas"][u]).days

 
'''Se excluyen pacientes que fueron diagnosticados antes de tener sintomas'''
df= df[df["Dias en Diagnosticar"] >= 0]


'''Se eliminan los espacios del campo estados del paciente'''
df["Estado"].unique()
df["Estado"]=[i.replace(" ","") for i in df["Estado"]]# Armando una lista que corrige el error de digitación


'''Se excluyen los pacientes que fallecen por causa diferente a COVID'''
df = df[df["Estado"] != "FallecidoNoaplicaNocausaDirecta"]


'''Los datos que estan en años no se modifican'''
df1 = df[df['Unidad de medida de la edad'] == 1]
df1.index = list(range(len(df1)))


'''Las edades que estan en meses se pasan a años'''
df2 = df[df['Unidad de medida de la edad'] == 2]
df2.index = list(range(len(df2)))
for t in range (len(df2['Unidad de medida de la edad'] == 2)):
    df2['Edad'][t] = df2['Edad'][t]/12


'''Las edades que estan en dias se pasan a años'''
df3 = df[df['Unidad de medida de la edad'] == 3]
df3.index = list(range(len(df3)))
for d in range (len(df3['Unidad de medida de la edad'] == 3)):
    df3['Edad'][d] = df3['Edad'][d]/360


'''Se unen las 3 bases'''
df = pd.concat([df1, df2, df3], axis=0)








def Edad(diagnostico):
    dx_Filtro=df[df["Edad"]>=diagnostico[0]][df[df["Edad"]>=diagnostico[0]]["Edad"]<=diagnostico[1]]
    Casos=round(dx_Filtro.groupby("Localidad de residencia").mean(),ndigits=2)
    # Casos=Edad_Filtro.groupby("Localidad de residencia").count()
    Casos=Casos.sort_values("Dias en Diagnosticar")
    fig, ax = plt.subplots(figsize=(10, 8))
    # Una función para poner la cantidad de casos
    def autolabel(rects):
        for rect in rects:
            width = rect.get_width()
            ax.annotate('{}'.format(width),
                        xy=(width,rect.get_y() + rect.get_height() / 2),
                        xytext=(3,0),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='left', va='center')

    rects=ax.barh(Casos.index,Casos["Dias en Diagnosticar"], color = '#ED671C')
    ax.set(xlim=(0, max(Casos["Dias en Diagnosticar"])+2))
    plt.title("El tiempo promedio en diagnosticar un paciente\n despues de presentar sintomas es de  "+
              str(round((Casos["Dias en Diagnosticar"]).mean(), ndigits = 2))+" dias")
    autolabel(rects)
    plt.text(0,-3,'Se realiza conversion a años para los pacientes que registraban unidad de medida en dias o meses ')
    plt.show()
    return

wid=widgets.IntRangeSlider(
    value=[min(df['Edad']), max(df['Edad'])],
    min=min(df['Edad']),
    max=max(df['Edad']),
    step=1,
    description='Edades:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    continue_update=False,
)
W1 = interactive(Edad,diagnostico=wid)



def Sintomas_dx(diagnostico):
    dx_Filtro=df[df["Dias en Diagnosticar"]>=diagnostico[0]][df[df["Dias en Diagnosticar"]>=diagnostico[0]]["Dias en Diagnosticar"]<=diagnostico[1]]
    Casos=round(dx_Filtro.groupby("Estado").count(),ndigits=2)
    # Casos=Edad_Filtro.groupby("Localidad de residencia").count()
    Casos=Casos.sort_values("Dias en Diagnosticar")
    fig, ax = plt.subplots(figsize=(12, 10))
    # Una función para poner la cantidad de casos
    def autolabel(rects):
        for rect in rects:
            width = rect.get_width()
            ax.annotate('{}'.format(width),
                        xy=(width,rect.get_y() + rect.get_height() / 2),
                        xytext=(3,0),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='left', va='center')

    rects=ax.barh(Casos.index,Casos["Dias en Diagnosticar"], color = 'orange', )
    ax.set(xlim=(0, max(Casos["Dias en Diagnosticar"])+2))
    plt.title(str(round((Casos["Dias en Diagnosticar"]).sum(), ndigits = 2)) + " pacientes contagiados por COVID-19")
    autolabel(rects)
    plt.show()
    return

wid=widgets.IntRangeSlider(
    value=[min(df['Dias en Diagnosticar']), max(df['Dias en Diagnosticar'])],
    min=min(df['Dias en Diagnosticar']),
    max=max(df['Dias en Diagnosticar']),
    step=1,
    description='Dias entre los sintomas y el diagnostico:',
    orientation='Horizontal',
    readout=True,
    readout_format='d',
    continue_update=False
)
W2 = interactive(Sintomas_dx,diagnostico=wid)


#### Dashboard

def Dashboard_COVID19():
    
    display(HTML('<h2>Dias promedio en diagnosticar un paciente despues de tener sintomas</h2>'+
                '<p>Seleccione un rango de edades:</p>'))
    display(W1)

    
    display(HTML('<h2>Distribucion de estados de pacientes de acuerdo a un rango de edad</h2>'+
                '<p>Seleccione un rango de edad y verifique :</p>'))
    display(W2)
    
    
    
    
    return

C:\Users\57321\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\57321\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\57321\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\57321\anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A valu

In [2]:
Dashboard_COVID19()

interactive(children=(IntRangeSlider(value=(0, 104), description='Edades:', max=104), Output()), _dom_classes=…

interactive(children=(IntRangeSlider(value=(0, 120), description='Dias entre los sintomas y el diagnostico:', …

In [3]:
df

,Fecha de inicio de síntomas,Fecha de diagnóstico,Ciudad de residencia,Localidad de residencia,Edad,Unidad de medida de la edad,Sexo,Tipo de caso,Ubicación,Estado,Dias en Diagnosticar
0,2020-02-26 00:00:00,2020-03-06 00:00:00,Bogotá,Usaquén,19.000000,1.0,F,Importado,Casa,Recuperado,9
1,2020-03-04 00:00:00,2020-03-10 00:00:00,Bogotá,Usaquén,22.000000,1.0,F,Importado,Casa,Recuperado,6
2,2020-03-07 00:00:00,2020-03-10 00:00:00,Bogotá,Engativá,28.000000,1.0,F,Importado,Casa,Recuperado,3
3,2020-03-06 00:00:00,2020-03-12 00:00:00,Bogotá,Fontibón,36.000000,1.0,F,Importado,Casa,Recuperado,6
4,2020-03-06 00:00:00,2020-03-12 00:00:00,Bogotá,Kennedy,42.000000,1.0,F,Importado,Casa,Recuperado,6
...,...,...,...,...,...,...,...,...,...,...,...
88,2020-08-13 00:00:00,2020-08-15 00:00:00,Bogotá,Engativá,0.044444,3.0,F,En estudio,Casa,Leve,2
89,2020-08-12 00:00:00,2020-08-15 00:00:00,Bogotá,Engativá,0.013889,3.0,M,En estudio,Casa,Leve,3
90,2020-08-14 00:00:00,2020-08-18 00:00:00,Bogotá,Bosa,0.013889,3.0,F,En estudio,Hospital,Moderado,4
91,2020-08-01 00:00:00,2020-08-19 00:00:00,Bogotá,Bosa,0.080556,3.0,M,En estudio,Hospital,Moderado,18
